<a href="https://colab.research.google.com/github/vizcayal/Thesis_UT/blob/V1/THESIS_MODEL_V1_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Installing packages needed**

In [15]:
!pip install gekko
!pip install ipynb


**Importing libraries needed**

In [16]:
import numpy as np
import time
from multiprocessing import *
from gekko import GEKKO
import pandas as pd
import os
from google.colab import files
from google.colab import drive
drive.mount("/content/drive")
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/')
from library import *


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**SAWMILL CLASS**

In [47]:
class Inventory:
  def __init__(self, Av, Dist, Wt=-1, Pt=-1):
    self.AcumResidues = []  #Residues generated by harvesting
    self.Available = list()
    self.Available.append(Av.copy()) #Dataframe that contains Available wood Inventory at diferent time t
    self.Available_Mill = [] #Dataframe that contains the Available Inventory close the mill to be harvested
    self.Candidates=[] #Candidate location for biorefinery
    self.RefCost = [] #Matrix Distance from any Mills and candidates cells to other cells
    self.Dist = Dist #Matrix Distance from any Mills and candidates cells to other cells
    self.Dist_Mill = [] #Dataframe that contains the Distance to the mill to be harvested
    self.Dist_Refi = [] #Dataframe that contains the Distance to the mill to be harvested
    self.Growth = [] #Growth ammount for each hexagon (feet cubic) 
    self.Harvested = [] #Ammount of harvested wood
    self.Harvested_Res = []  #Residues Harvested
    self.Mills = [] #Dataframe that contains Id, typewood, type_product, and demand of mills
    self.ProdType = Pt #type of product generated (pulp or saw)
    self.Radio_Mill = 15
    self.Radio_Refinery = 70
    self.Residues = []  #Residues generated by harvesting
    self.WoodType = Wt #type of wood needed (softwood, hardwood)
    
  def Harvest_Mill_Close(self,t, j):
    #Given a period t and a Mill j the function provide the harvested wood and substract it from available inventory
    m = GEKKO(remote=True)
    m.options.solver = 3 #IPOPT solver
    m.options.IMODE = 3 #steady state optimization
    Id1 =self.Mills.loc[j,('Unique_ID')]
    a =self.Cell_Less(Id1,t,self.Radio_Mill)
    self.Dist_Mill = a.loc[:, ('DEST','DIST')]
    Num = len(self.Dist_Mill)
    H = m.Array(m.Var,(Num),value = 0)
    
    Dem = self.Mills.loc[j,('Demand')]
    self.Available_Mill = self.Disp_Radio( Id1, t, self.Radio_Mill)
    
    for i in range(Num):
      H[i].lower = 0
      H[i].upper = self.Available_Mill[i]
      
    prod0 = m.Intermediate(m.sum([self.Dist_Mill.loc[i,('DIST')]*H[i] for i in range(Num)]))    
    m.Equation(Dem ==m.sum([H[i] for i in range(0,len(H))]))
    m.Obj(prod0) 
    m.solve(disp=False,debug=False)
    a = self.Dist_Mill
    flat = [item for l in H for item in l]
    a['Harvested'] = flat 
    ans = a

    #Storing Harvested 
    self.Harvested[t].loc[:,('Harvested')] = 0 
    print('Reseting stage:',t+1,' Mill: ',Id1,sum(self.Harvested[t].loc[:,("Harvested")]))   

    for j in range(len(ans)):                                                                         
      if not(self.Harvested[t][self.Harvested[t]["Unique_ID"]==ans.loc[j,('DEST')]].empty):
        a1 = self.Harvested[t]["Unique_ID"]==ans.loc[j,('DEST')]
        a2 = int(np.where(a1)[0])
        self.Harvested[t].loc[a2,("Harvested")] = ans.loc[j,('Harvested')]  
    print('Wood Harvested in stage:',t,sum(self.Harvested[t].loc[:,("Harvested")]))
    
    #Updating Available Inventory
    for j in range(len(ans)):                                                                         
      if not(self.Available[t+1][self.Available[t+1]["Unique_ID"]==ans.loc[j,('DEST')]].empty):
        a1 = self.Available[t+1]["Unique_ID"]==ans.loc[j,('DEST')]
        a2 = int(np.where(a1)[0])
        self.Available[t+1].loc[a2,("SumOfTotal")] = self.Available[t+1].loc[a2,("SumOfTotal")] - ans.loc[j,('Harvested')] 
    
    
  def Harvest_Mill_Dense(self,t, j):
    #Given a period t and a Mill j the function provide the harvested wood and substract it from available inventory
    m = GEKKO(remote=True)
    m.options.solver = 3 #IPOPT solver
    m.options.IMODE = 3 #steady state optimization
    Id1 =self.Mills.loc[j,('Unique_ID')] #look for mill in position j
    print('Mill ID', Id1)
    a =self.Cell_Less(Id1,t,self.Radio_Mill) #look for cells between the range Radio_Mill
    self.Dist_Mill = a.loc[:, ('DEST','DIST')] #fill Distances to mills
    Num = len(self.Dist_Mill)
    H = m.Array(m.Var,(Num),value = 0)
    Dem = self.Mills.loc[j,('Demand')]
    self.Available_Mill = self.Disp_Radio( Id1, t+1, self.Radio_Mill)

    D1 = np.sum(self.Available_Mill)
    print('Demanda:', Dem, 'Disponible',D1, 'Year: ',t, 'Mill: ',Id1 )
    
    for i in range(Num):
      H[i].lower = 0
      H[i].upper = self.Available_Mill[i]
      if (self.Available_Mill[i]<0):
        print('Negativo',self.Available_Mill[i], 'Year: ',t, 'Mill: ',j)
      
    prod0 = m.Intermediate(-m.sum([self.Dist_Mill.loc[i,('DIST')]*H[i] for i in range(Num)]))    
    m.Equation(Dem ==m.sum([H[i] for i in range(0,len(H))]))
    m.Obj(prod0) 
    m.solve(disp=False,debug=False)
    a = self.Dist_Mill
    flat = [item for l in H for item in l]
    a['Harvested'] = flat  
    ans = a

    #Storing Harvested 
    for j in range(len(ans)):                                                                         
      if not(self.Harvested[t][self.Harvested[t]["Unique_ID"]==ans.loc[j,('DEST')]].empty):
        a1 = self.Harvested[t]["Unique_ID"]==ans.loc[j,('DEST')]
        a2 = int(np.where(a1)[0])
        self.Harvested[t].loc[a2,("Harvested")] = self.Harvested[t].loc[a2,("Harvested")] + ans.loc[j,('Harvested')]  

    
    #Updating Available Inventory
    for j in range(len(ans)):                                                                         
      if not(self.Available[t+1][self.Available[t+1]["Unique_ID"]==ans.loc[j,('DEST')]].empty):
        a1 = self.Available[t+1]["Unique_ID"]==ans.loc[j,('DEST')]
        a2 = int(np.where(a1)[0])
        self.Available[t+1].loc[a2,("SumOfTotal")] = self.Available[t+1].loc[a2,("SumOfTotal")] - ans.loc[j,('Harvested')] 
        l0  = self.Available[t+1]["SumOfTotal"]<0
        if np.sum(self.Available[t+1].loc[:,("SumOfTotal")])<0 :
          print('****Available Negative')
    
    print('Cells Harvested by mill: ',Id1,'\n',self.Harvested[t][self.Harvested[t]["Harvested"]>0])
    print('Total Harvested',np.sum(self.Harvested[t].loc[:,("Harvested")])) 

  def Harvest_Mill_Close1(self,t, j):
    #Given a period t and a Mill j the function provide the harvested wood and substract it from available inventory
    m = GEKKO(remote=True)
    m.options.solver = 3 #IPOPT solver
    m.options.IMODE = 3 #steady state optimization
    Id1 =self.Mills.loc[j,('Unique_ID')] #look for mill in position j
    a =self.Cell_Less(Id1,t,self.Radio_Mill) #look for cells between the range Radio_Mill
    self.Dist_Mill = a.loc[:, ('DEST','DIST')] #fill Distances to mills
    Num = len(self.Dist_Mill)
    H = m.Array(m.Var,(Num),value = 0)
    Dem = self.Mills.loc[j,('Demand')]
    self.Available_Mill = self.Disp_Radio( Id1, t+1, self.Radio_Mill)

    D1 = np.sum(self.Available_Mill)
    print('Demanda:', Dem, 'Disponible',D1, 'Year: ',t, 'Mill: ',Id1 )
    
    for i in range(Num):
      H[i].lower = 0
      H[i].upper = self.Available_Mill[i]
      if (self.Available_Mill[i]<0):
        print('Negativo',self.Available_Mill[i], 'Year: ',t, 'Mill: ',j)
      
    prod0 = m.Intermediate(m.sum([self.Dist_Mill.loc[i,('DIST')]*H[i] for i in range(Num)]))    
    m.Equation(Dem ==m.sum([H[i] for i in range(0,len(H))]))
    m.Obj(prod0) 
    m.solve(disp=False,debug=False)
    a = self.Dist_Mill
    flat = [item for l in H for item in l]
    a['Harvested'] = flat  
    ans = a

    #Storing Harvested 
    for j in range(len(ans)):                                                                         
      if not(self.Harvested[t][self.Harvested[t]["Unique_ID"]==ans.loc[j,('DEST')]].empty):
        a1 = self.Harvested[t]["Unique_ID"]==ans.loc[j,('DEST')]
        a2 = int(np.where(a1)[0])
        self.Harvested[t].loc[a2,("Harvested")] = self.Harvested[t].loc[a2,("Harvested")] + ans.loc[j,('Harvested')]  

    
    #Updating Available Inventory
    for j in range(len(ans)):                                                                         
      if not(self.Available[t+1][self.Available[t+1]["Unique_ID"]==ans.loc[j,('DEST')]].empty):
        a1 = self.Available[t+1]["Unique_ID"]==ans.loc[j,('DEST')]
        a2 = int(np.where(a1)[0])
        self.Available[t+1].loc[a2,("SumOfTotal")] = self.Available[t+1].loc[a2,("SumOfTotal")] - ans.loc[j,('Harvested')] 
        l0  = self.Available[t+1]["SumOfTotal"]<0
        if np.sum(self.Available[t+1].loc[:,("SumOfTotal")])<0 :
          print('****Available Negative')
    
    print('Cells Harvested by mill: ',Id1,'\n',self.Harvested[t][self.Harvested[t]["Harvested"]>0])
    print('Total Harvested',np.sum(self.Harvested[t].loc[:,("Harvested")])) 

  
  def Harvest_All_Mills(self,t,k):
  #Harvest k mills at year t
    print('Stage:',t)
    self.Available.append(self.Available[t].copy())
    #print('Harvested:', self.Harvested)
    for i in range(k):
      print('Cosechando Sawmill:',i)
      self.Harvest_Mill_Dense(t,i)
    
    self.Residues.append(self.Harvested[t].loc[:,('Unique_ID','Harvested')].copy())
    self.Harvested.append(self.Harvested[t].copy()) 
    self.Harvested[t+1].loc[:,('Harvested')] = 0
    self.Residues[t].loc[:,('Harvested')] = round(self.Residues[t].loc[:,('Harvested')]*0.05)
    
  
  def Inventory_Growth(self,t,rate = 0):
    #increase the available inventory of period t
    if (rate == 0):
      r = self.Growth
    else:
      r = rate
    self.Available[t].loc[:,("SumOfTotal")] = self.Available[t].loc[:,("SumOfTotal")]*(1+r/100)

  
  def Cell_Less(self, Id_Mill, t, Radio=20):
    #Given Time, Radio and a Mill ID this function returns a list of cells with Distance and Availability
    Ans = self.Dist[(self.Dist['ORIG'] == Id_Mill)&(self.Dist['DIST'] <= Radio)] #CELLS EQUAL ID_MILL AND CLOSER THAN RADIO
    rows = Ans.shape[0]
    ind = [i for i in range(rows)]
    Ans.index=ind    
    Disp =[]

    for j in range(len(Ans)):
      if self.Available[t][self.Available[t]["Unique_ID"]==Ans.loc[j,('DEST')]].empty:
        Disp.append(0)
      else:
        Disp.append(int(self.Available[t][self.Available[t]["Unique_ID"]==Ans.loc[j,('DEST')]].SumOfTotal.copy()))
    Ans["AVAIL"] = Disp 
    return(Ans)


  def Cell_Less_Res(self, Id_Mill, t, Radio=50):
    #Given Time, Radio and a Mill ID this function returns a list of cells with Distance and Forest Residues available
    Ans = self.Dist[(self.Dist['ORIG'] == Id_Mill)&(self.Dist['DIST'] <= self.Radio_Refinery)] #CELLS EQUAL ID_MILL AND CLOSER THAN RADIO
    rows = Ans.shape[0]
    ind = [i for i in range(rows)]
    Ans.index=ind    
    Disp =[]

    for j in range(len(Ans)):                                                                         
      if self.Residues[t][self.Residues[t]["Unique_ID"]==Ans.loc[j,('DEST')]].empty:
        Disp.append(0)
      else:
        Disp.append(int(self.Residues[t][self.Residues[t]["Unique_ID"]==Ans.loc[j,('DEST')]].Harvested))
    Ans["AVAIL"] = Disp 
    return(Ans)


  def Disp_Radio(self, Id_Mill, t, Radio=20):
    #Given a Time and Radio and a Mill ID this function returns a list of Inventory Available [Unique_ID, Amount_Available]
    Ans = self.Dist[(self.Dist['ORIG'] == Id_Mill)&(self.Dist['DIST'] <= Radio)] #CELLS EQUAL ID_MILL AND CLOSER THAN RADIO
    rows = Ans.shape[0]
    ind = [i for i in range(rows)]
    Ans.index=ind    
    Disp =[]
    for j in range(len(Ans)):                                                                         
      if self.Available[t][self.Available[t]["Unique_ID"]==Ans.loc[j,('DEST')]].empty:
        Disp.append(0)
      else:
        Disp.append(int(self.Available[t][self.Available[t]["Unique_ID"]==Ans.loc[j,('DEST')]].SumOfTotal))
    return(Disp)
  

  def Num_Cell(self, Id_Mill, Radio=10):
    #Given a Radio and a Mill ID this function returns cell among the radio distance
    Ans = self.Dist[(self.Dist['ORIG'] == Id_Mill)&(self.Dist['DIST'] <= Radio)]
    return(len(Ans))
  
  
  def Mill_Update(self):
    i1 = self.Mills['s_h'] == self.WoodType
    self.Mills = self.Mills.loc[i1,:]
    Num = len(self.Mills)
    ind = [i for i in range(Num)]
    print('Num',Num)
    self.Mills.index = ind
    print(self.Mills)

  def Disp_Radio_Res(self, Id_Mill, t, Radio=20):
    #Given a Time and Radio and a Mill ID this function returns a list of Residues Available [Unique_ID, Amount_Available]
    Ans = self.Dist[(self.Dist['ORIG'] == Id_Mill)&(self.Dist['DIST'] <= self.Radio_Refinery)] #CELLS EQUAL ID_MILL AND CLOSER THAN RADIO
    rows = Ans.shape[0]
    ind = [i for i in range(rows)]
    Ans.index=ind    #update index from 0 to rows-1
    Disp =[]
    for j in range(len(Ans)):  
      #print(self.AcumResidues[t]["Unique_ID"]==Ans.loc[j,('DEST')])                                                   
      if self.AcumResidues[t][self.AcumResidues[t]["Unique_ID"]==Ans.loc[j,('DEST')]].empty:
        Disp.append(0)
      else:
        Disp.append(int(self.AcumResidues[t][self.AcumResidues[t]["Unique_ID"]==Ans.loc[j,('DEST')]].SumOfTotal))
    return(Disp)


  def Harvest_Refinery(self,t,Cand, Demanda=20000): #Evaluates Candidate location for refinery in year t
    m2 = GEKKO(remote=True)
    m2.options.solver = 3 #IPOPT solver
    m2.options.IMODE = 3 #steady state optimization

    Id1 =self.Candidates[Cand] #assign unique ID
    a =self.Cell_Less_Res(Id1,t,self.Radio_Refinery) #look for hexagons between radio distance
    self.Dist_Refi = a.loc[:, ('DEST','DIST')]  #obtain column Destination and Distance from 'a'
    Num = len(self.Dist_Refi) #calculate the number of cells betwee 
    H = m2.Array(m2.Var,(Num),value = 0)

    self.Available_Refi = self.Disp_Radio_Res(Id1, t+1, self.Radio_Refinery) #look for available quantity in cells
      
    for i in range(Num):
      H[i].lower = 0
      H[i].upper = self.Available_Refi[i]  
    prod0 = m2.Intermediate(m2.sum([self.Dist_Refi.loc[i,('DIST')]*H[i] for i in range(Num)]))
    m2.Equation(Demanda ==m2.sum([H[i] for i in range(0,len(H))]))
    m2.Obj(prod0) 
    m2.solve(disp=False,debug=False)
    
    a = self.Dist_Refi
    flat = [item for l in H for item in l]
    a['Harvested'] = flat
    ans = a
        
    #Storing Harvested 
    for j in range(len(ans)):                                                                         
      if not(self.Harvested_Res[t][self.Harvested_Res[t]["Unique_ID"]==ans.loc[j,('DEST')]].empty):
        a1 = self.Harvested_Res[t]["Unique_ID"]==ans.loc[j,('DEST')]
        a2 = int(np.where(a1)[0])
        self.Harvested_Res[t].loc[a2,("Harvested")] = self.Harvested_Res[t].loc[a2,("Harvested")] + ans.loc[j,('Harvested')]  
    
    #Updating Available Inventory
    print('Residues Available Before',np.sum(self.AcumResidues[t+1].loc[:,("SumOfTotal")]))     
    
    for j in range(len(ans)):                                                                         
      if not(self.AcumResidues[t+1][self.AcumResidues[t+1]["Unique_ID"]==ans.loc[j,('DEST')]].empty):
        a1 = self.AcumResidues[t+1]["Unique_ID"]==ans.loc[j,('DEST')]
        a2 = int(np.where(a1)[0])
        self.AcumResidues[t+1].loc[a2,("SumOfTotal")] = self.AcumResidues[t+1].loc[a2,("SumOfTotal")] - ans.loc[j,('Harvested')] 
    
        l0  = self.AcumResidues[t+1]["SumOfTotal"]<0
        if np.sum(self.Available[t+1].loc[:,("SumOfTotal")])<0 :
          print('****Available Negative')
    
    print('answer\n',ans)

    print('Cells Harvested',self.Harvested_Res[t][self.Harvested_Res[t]["Harvested"]>0]) 
    print('Distances',self.Dist_Refi.loc[self.Harvested_Res[t]["Harvested"]>0],('DIST')) 
    
    
    print('Total Harvested',np.sum(self.Harvested_Res[t].loc[:,("Harvested")])) 
    
    print('Residues Available After',np.sum(self.AcumResidues[t+1].loc[:,("SumOfTotal")])) 


    res = round(np.dot(flat,self.Dist_Refi.loc[:,('DIST')]))  #Delivery cost to refinery
    self.RefCost[Cand] += res
    print('Cost', self.RefCost[Cand])
  

  def Optimal_Location_Total(self,Y,NCan = 0):
    #run Ncan locations from year 0 to year 'y'
    if (NCan == 0):
      Cans = len(self.Candidates)
    else:
      Cans = NCan
    self.RefCost = [0]*Cans

    for c in range(Cans):
      self.AcumResidues = []
      self.AcumResidues.append(self.Residues[0].copy())
      self.AcumResidues[0].columns=["Unique_ID", "SumOfTotal"]
      self.AcumResidues[0].loc[:,('SumOfTotal')] = 0
        
      for i in range(Y):
        self.AcumResidues.append(self.AcumResidues[i].copy())
        self.AcumResidues[i+1].columns=["Unique_ID", "SumOfTotal"]
        self.AcumResidues[i+1]['SumOfTotal'] =self.AcumResidues[i+1]['SumOfTotal'] + self.Residues[i]['Harvested']
        self.Harvest_Refinery(i,c)
        self.Harvested_Res.append(self.Harvested_Res[i].copy())     
        self.Harvested_Res[i+1].loc[:,('Harvested')] = 0
        

      
 
    

**UPLOADING FOREST INVENTORY DATA**

**1ST PHASE - MILLS DEMAND - SHORTEST DISTANCE CRITERIA**

In [67]:
# Main Program
start_time = time.time()
#Upload distances from each hexagon to other
#Hardwood_Sawlog_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Hardwood_Sawlog.csv")
Hardwood_Sawlog_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Hardwood_Saw_V2.csv")
Hardwood_Pulp_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Hardwood_Pulp_V2.csv")
Softwood_Sawlog_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Softwood_Saw_V2.csv")
Softwood_Pulp_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Softwood_Pulp_V2.csv")
Dist_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Dist_Mill.csv")
Mills_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Mills_TN.csv")


i1 = Mills_Data['plant'] == "Composite Panel"
Mills_Data.loc[i1,('Demand')] = Mills_Data.loc[i1,('Demand')]*128 #transforming cords to cubic feet
i1 = Mills_Data['plant'] == "Pulpmill"
Mills_Data.loc[i1,('Demand')] = Mills_Data.loc[i1,('Demand')]*128 #transforming cords to cubic feet
i1 = Mills_Data['plant'] == "Sawmill"
Mills_Data.loc[i1,('Demand')] = Mills_Data.loc[i1,('Demand')]*83.3  #transforming mbf to cubic feet
i1 = Mills_Data['plant'] == "Venner"
Mills_Data.loc[i1,('Demand')] = 133280 #transforming mbf to cubic feet

Mills_Data[Mills_Data.Unique_ID==86399]
Mills_Data.drop(index=106, inplace=True)

b = Mills_Data['s_h'] =='H'
a =Mills_Data[b]
Num2 = len(a)   
ind = [i for i in range(Num2)]
a.index = ind
z =a[a['plant'] == 'Sawmill']
Num3 = len(z)   
ind = [i for i in range(Num3)]
z.index = ind

#print(list(Dist_Data['ORIG']))

YEARS = 1
#Initializing Inventories
Hardwood_Sawlog = Inventory(Hardwood_Sawlog_Data,Dist_Data,Wt='H')
#Hardwood_Sawlog.Dist[(Hardwood_Sawlog.Dist['ORIG'] == 92122)&(Hardwood_Sawlog.Dist['DIST'] <= 20)]
#Hardwood_Sawlog_Data.Cell_Less(92122,0)
Hardwood_Sawlog.Available[0][Hardwood_Sawlog.Available[0]["Unique_ID"]==90913]
#Hardwood_Pulp = Inventory(Hardwood_Pulp_Data,Dist_Data,Wt='H')
#Softwood_Sawlog = Inventory(Softwood_Sawlog_Data,Dist_Data,Wt='S')
#Softwood_Pulp = Inventory(Softwood_Pulp_Data,Dist_Data,Wt='S')

r1 = len(Hardwood_Sawlog.Available[0])
Id_li = list(Hardwood_Sawlog.Available[0].loc[:,("Unique_ID")])
Ha_li =[0]*r1
Data = {'Unique_ID':Id_li,'Harvested':Ha_li}
Hardwood_Sawlog.Harvested.append(pd.DataFrame(Data).copy())

Num_Mills = len(z)
#Num_Mills = 1

ma = Dist_Data['ORIG'].unique()
Hardwood_Sawlog.Candidates = ma
Num_Cand = len(ma)

#if (86399 in Dist_Data['ORIG']):
#print(Dist_Data[Dist_Data['ORIG']==40476)
#Assigning Candidates Id to Mills Id while getting the mills exact distance
#for k in range(Num_Mills):
  #z.loc[k,('Unique_ID')] = ma[k % Num_Cand]
  
print('Mills',Mills_Data.loc[:,])

Hardwood_Sawlog.Mills=z.loc[:,]

#Harvest all years all mills
for y in range(YEARS):
  Hardwood_Sawlog.Harvest_All_Mills(y,Num_Mills)    

for y in range(YEARS):
  Hardwood_Sawlog.Harvested[y].to_csv('/content/drive/My Drive/Colab Notebooks/Results/year'+str(y)+'.csv')

#Totalize the Wood Harvested 
Cs =len(Hardwood_Sawlog.Harvested[YEARS])
for y in range(YEARS):
  for i in range(Cs):
    Hardwood_Sawlog.Harvested[YEARS].loc[i,('Harvested')] = Hardwood_Sawlog.Harvested[YEARS].loc[i,('Harvested')] + Hardwood_Sawlog.Harvested[y].loc[i,('Harvested')] 

Hardwood_Sawlog.Harvested[YEARS].to_csv('/content/drive/My Drive/Colab Notebooks/Results/year'+str(YEARS)+'.csv')

print ("*********1ST STAGE TOOK", time.time() - start_time, "TO RUN")

       Unique_ID  SumOfTotal  Shape_Area
0         105516      103943         5.0
1         110795      636099         5.0
2          95740      220802         5.0
3         117121     2248222         5.0
4          81822      529145         5.0
...          ...         ...         ...
18752     101249      343820         5.0
18753      98245      645912         5.0
18754      75320       21323         5.0
18755      82055     1400398         5.0
18756      59425     1097654         5.0

[18757 rows x 3 columns]


**SECOND STAGE**

In [ ]:
start_time = time.time()
Data = {'Unique_ID':Id_li,'Harvested':Ha_li}
Hardwood_Sawlog.Harvested_Res.append(pd.DataFrame(Data).copy())
Hardwood_Sawlog.Optimal_Location_Total(YEARS,2)

print ("*********2ND STAGE TOOK", time.time() - start_time, "TO RUN")